In [1]:
from aiida import orm, load_profile

#load_profile('dev')

import numpy

from aiida import load_profile
load_profile() 
from aiida.orm import QueryBuilder, RemoteData, load_node
from aiida.orm import utils,Code, Computer, Data, Node, StructureData,Group, load_code, Str
from aiida.plugins import CalculationFactory,DataFactory
from aiida.orm.nodes.data.upf import get_pseudos_from_structure
from aiida.engine import submit

from aiida.plugins import WorkflowFactory

from aiida_quantumespresso_ph.workflows.dynamical_matrix import DynamicalMatrixWorkChain
from aiida_quantumespresso_ph.workflows.ph_interpolate import PhInterpolateWorkChain

In [2]:
A_sites = ['Ba']
B_site = ['Ti']
a_arr=[3.977]
for a_site in A_sites:
    name = a_site[0]+B_site[0]+'O'

    #UNCOMMENT to create a group
    #slabel = 'structures/PhononTest'
    #gstruct = Group(label=slabel).store()
    

    for a in a_arr:
        StructureData = DataFactory("structure")
        alat =  a # angstrom
        cell = [[alat, 0., 0.,],
                [0., alat, 0.,],
                [0., 0., alat,]]
        s = StructureData(cell=cell)
        s.append_atom(position=(alat/2.,alat/2.,alat/2.), symbols=a_site)
        s.append_atom(position=(0.,0.,0.), symbols=B_site)
        s.append_atom(position=(alat/2.,0.,0.), symbols='O')
        s.append_atom(position=(0.,alat/2.,0.), symbols='O')
        s.append_atom(position=(0.,0.,alat/2.), symbols='O')
        
        #UNCOMMENT to store
        #snode=s.store()
        #gstruct.add_nodes([snode])

In [3]:
codename_pw = 'pw-sirius-module@daint-s1073'
codename_ph =  'ph-fromsiriusmod@daint-s1073'
codename_q2r = 'qe-6.4.1_q2r@theospc2'
codename_matdyn = 'qe-6.4.1_matdyn@theospc2'

In [4]:
from aiida_quantumespresso.common.types import ElectronicType, RelaxType


In [5]:
Dict = DataFactory('dict')
param_pw = {
            'CONTROL': {
                'calculation': 'scf',
                'restart_mode': 'from_scratch',
                'verbosity': 'high',
                'wf_collect': True,
                'tprnfor': True,
            },
            'SYSTEM': {
                'ecutwfc': 60, # 100,
                'ecutrho': 600, #1000,
                'occupations': 'fixed',
            },
            'ELECTRONS': {
                'conv_thr': 1.e-12, #1.e-12,
}}

param_rel = {
            'CONTROL': {
                'calculation': 'relax',
                'restart_mode': 'from_scratch',
                'verbosity': 'high',
                'wf_collect': True,
                'tprnfor': True,
            },
            'SYSTEM': {
                'ecutwfc': 60, # 100,
                'ecutrho': 600, #1000,
                'occupations': 'fixed',
            },
            'ELECTRONS': {
                'conv_thr': 1.e-12, #1.e-12,
}}

KpointsData = DataFactory('array.kpoints')
kpoints = KpointsData()
kpoints.set_kpoints_mesh([8,8,8])

over = {}
over['relax'] = {'base':{'kpoints_distance':0.2,'pw': {'parameters': param_pw, 'metadata': {'options': { 'max_wallclock_seconds': 600}}},'pseudo_family': 'SSSP/1.1/PBEsol/precision'}}

#over['pw_base'] = {'kpoints_distance':0.2,'pw': {'parameters': param_pw, 'metadata': {'options': { 'max_wallclock_seconds': 600}}},'pseudo_family': 'SSSP/1.1/PBEsol/precision'}
over['ph_base'] = {'ph': {'parameters':{'INPUTPH':{'tr2_ph':1.0e-2}}, 'qpoints': [1,1,1], 'metadata': {'options': { 'max_wallclock_seconds': 60*5, 'queue_name':'debug', 'resources': {'num_machines': 2}}}},'pseudo_family': 'SSSP/1.1/PBEsol/precision'}

In [14]:
builder = DynamicalMatrixWorkChain.get_builder_from_protocol(
    pw_code=codename_pw,
    ph_code=codename_ph,
    structure=s,
    electronic_type=ElectronicType.INSULATOR, 
    overrides=over,
    relax_type = RelaxType.POSITIONS,
)
builder

Process class: DynamicalMatrixWorkChain
Inputs:
clean_workdir: true
metadata: {}
ph_base:
  metadata: {}
  ph:
    code: Quantum ESPRESSO ph.x from Sirius Module on Daint
    metadata:
      options:
        max_wallclock_seconds: 300
        queue_name: debug
        resources:
          num_machines: 2
          num_mpiprocs_per_machine: 12
        stash: {}
        withmpi: true
    parameters:
      INPUTPH:
        epsil: true
        tr2_ph: 0.01
    qpoints: 'Kpoints mesh: 1x1x1 (+0.0,0.0,0.0)'
relax:
  base:
    kpoints_distance: 0.2
    kpoints_force_parity: false
    metadata: {}
    pw:
      code: Quantum ESPRESSO pw.x SIRIUS  module on Daint
      metadata:
        options:
          max_wallclock_seconds: 600
          resources:
            num_machines: 1
          stash: {}
          withmpi: true
      parameters:
        CONTROL:
          calculation: relax
          etot_conv_thr: 5.0e-05
          forc_conv_thr: 0.0001
          restart_mode: from_scratch
        

In [16]:
from aiida.engine import submit

submit(builder)

<WorkChainNode: uuid: f689dda8-2f53-48d7-8006-263635c422de (pk: 1686458) (aiida.workflows:quantumespresso.dynamical_matrix)>

In [4]:
from aiida.orm import load_node
resWF= [1225694]
dm_node = load_node(1225694)
Dict = DataFactory('dict')

codename_q2r = 'qe-6.4.1_q2r@theospc2'
codename_matdyn = 'qe-6.4.1_matdyn@theospc2'
KpointsData = DataFactory('array.kpoints')
qpoints = KpointsData()
qpoints.set_kpoints_mesh([2,2,2])


param_q2r = Dict(dict={
    'INPUT': {"zasr":"simple"}, #necessary as this is an insulator (LO-TO splitting), "crystal" would also work here
})

param_md = Dict(dict={
    'INPUT': {"asr":"simple"},
    })
d_qpoints = KpointsData()

n=30
qGX=[[0,0,i] for i in numpy.linspace(0,.5,n)]
qXM=[[0,i,.5] for i in numpy.linspace(0,.5,n)]
qMR=[[i,.5,.5] for i in numpy.linspace(0,.5,n)]
qRG=[[i,i,i] for i in numpy.linspace(.5,0,n)]
qGM=[[0,i,i] for i in numpy.linspace(0,.5,n)]
d_qpoints.set_kpoints(qGX[:-1]+qXM[:-1]+qMR[:-1]+qRG[:-1] + qGM)

In [31]:

code = load_code(codename_q2r)
ret_folder = dm_node.outputs.ph_retrieved
builder = code.get_builder()
builder.parameters = param_q2r
builder.parent_folder = ret_folder
builder.metadata.label = "q2r for calc " 


builder.metadata.options.resources = {'num_machines': 2,'num_cores_per_mpiproc':1}
builder.metadata.options.max_wallclock_seconds = 60 * 60

child = submit(builder)

In [27]:
dm_node.outputs.ph_retrieved 

<FolderData: uuid: 70556cd5-8b4b-440e-8167-88623b11e71c (pk: 1225713)>

In [29]:

ret_folder = dm_node.outputs.ph_retrieved #load_node(n)
inputs = {
    'dynmat_folder': ret_folder,
    'q2r': {
        'q2r': {
            'parameters':  param_q2r,
            'metadata': {
                'options': {
                    'resources': {'num_machines': 1,'num_cores_per_mpiproc':1},
                    'max_wallclock_seconds': 2*60
                },
                'description': 'phono_disp_test q2r'
            },
            'code': load_code(codename_q2r),
        },
    },
    'matdyn': {
        'matdyn': {
            'parameters':  param_md,
            'kpoints': d_qpoints,
            'metadata': {
                'options': {
                    'resources': {'num_machines': 1,'num_cores_per_mpiproc':1},
                    'max_wallclock_seconds': 2*60
                },
                'description': 'phono_disp_test matdyrn'
            },
            'code': load_code(codename_matdyn),
        },
    },
}
result = submit(PhInterpolateWorkChain, **inputs)


In [ ]:
ret_folder.

In [2]:
from aiida.plugins import DataFactory
FolderData = DataFactory('core.folder')

In [2]:
from aiida import load_profile
load_profile() 
from aiida.orm import FolderData
folder = FolderData()

In [ ]:
folder.base.repository.